In [1]:
################################################################################################
# <문장 임베딩 예시>
# sentence-bert와 Faiss 라이브러리를 이용하여 검색 MRR(Mean Reciprocal Rank)측정 예시 4
# - contexts(문서)들를 슬라이등 윈도우 방식으로 max 512로 나누고 Faiss에 임베딩 벡터를 추가한다.
# - 쿼리문장 임베딩벡터를 구해서 각 contexts(문서) Faiss 인덱스에서 sub 문장 벡터와 비교하여 유사도 스코어 계산.
# - 말뭉치는  KorQuAD_v1.0 파일과 aihub(aihub.or.kr) 뉴스 기사 기계독해 데이터 QuA 파일 3종류 이용
#------------------------------------------------------------------------------
# 1. 검색모델 로딩
# 2. JSON 파일 로딩 후 df 만듬
# 3. 문단을 슬라이딩 윈도우 기법으로 세션으로 분리
# 4. 세션별 문장들의 토큰 평균들에 대해 Faiss 임베딩 생성
# 5. 쿼리 후 가장 유사한 문장(문서)의 sub 문장 평균 구함
# 6. MRR 계산
#-------------------------------------------------------------------------------
# ***onnx 모델은 성능이 영 안나옴.(이유 모름)
#-------------------------------------------------------------------------------
# sklenarn 으로 cosine 확인 예제
# from sklearn.metrics.pairwise import cosine_similarity
# cosine = cosine_similarity([embed_querys[0]], [embed_querys_1[0]]) # (1,768) 식에 2차원 배열입력되어야 함.
#-------------------------------------------------------------------------------
################################################################################################

import faiss
import numpy as np
import pandas as pd
import time

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything, mlogging, bi_encoder

logger = mlogging(loggername="MRR-sliding", logfilename="../../../log/MRR-sliding")
device = GPU_info()
device = 'cpu'  # cpu 테스트 할때

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 500             # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5                # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
avg_num = 1                 # 쿼리에 대해 sub 문장들중 최대 scorce를 갖는 문장을 몇개 찾고 평균낼지.(3=쿼리에 가장 유사한 sub문장 3개를 찾고 평균을 냄)
faiss_index_method = 0      # 0= Cosine Similarity 적용(IndexFlatIP 사용), 1= Euclidean Distance 적용(IndexFlatL2 사용)

# 슬라이딩 윈도우 param
WINDOW_SIZE=256   # 문단을 몇 token으로 나눌지
#SLIDING_SIZE=64   # 중첩되는 token 

# 차원 축소 관련 param
# 차원 축소 할지, 768 값을 앞에 384 만큼만 배열 resize 시킴.  
# - 384로 줄일대 -2% 성능 저하 발생(512: -1.2%, 256: -6%)
DIM_RESIZE_METHOD = 0  # 0=차원축소 안함/1=resize 로 차원축소/2=Dense 모델 이용 차원축소
DIM_RESIZE_LEN = 128

# ONNX 모델 사용(*무조건 cpu로 실행됨)
IS_ONNX_MODEL = False        # True=onnx 모델 사용

#------------------------------------------------------------------------------------

# 토큰 임베딩 관련 param
IS_EMBED_DIVIDE = False     # False=토큰임베딩 안함/True=문장에 대해 토큰별루 분리해서 평균구해서 임베딩 구함
EMBED_DIVIDE_LEN = [5,7,9]#[1,1,1]#[5,7,9] #5 # 문장을 몇개(토큰)으로 분리할지 (7,8,10) 일때 성능 좋음=>50.8%, (5,7,9) 일때 차원축소 128=>41.80%(81.8%) 성능 좋음
MAX_TOKEN_LEN = 40 #512 #40          # 최대 몇개 token까지만 임베딩 할지

# 쿼리 임베딩 param
# - 쿼리 임베딩 안할때 보다도 성능이 떨어짐(문장검색 5%이상 떨어짐,단어검색은 유사함(10정도))
IS_QUERY_EMBED_DIVIDE = False  # True= 쿼리도 토큰으로 나누고 각 토큰별 max 값들 구한후 합해서 유사도 구함
QUERY_DIVIDE_LEN=10  # 쿼리를 몇개 토큰으로 나눠서 평균을 구할지 계수 설정(*10개 정도면 쿼리임베딩 안할때 단어검색일때 유사도 비슷하게 나옴)

# param 인자 범위 체크
if faiss_index_method > 1 or faiss_index_method < 0:
    raise ValueError(f"faiss_index_method = {faiss_index_method} is not bad!!")
    
seed_everything(seed)

logfilepath:../../../log/MRR-sliding_2023-03-27.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [2]:
#-------------------------------------------------------------------------------------
# 1. 검색모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
#-------------------------------------------------------------------------------------
import torch
from myutils import bi_encoder, dense_model, onnx_model, onnx_embed_text

bi_encoder_path = "bongsoo/klue-sbert-v1" #"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls

 # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 
out_dimension = 128 if DIM_RESIZE_METHOD == 2 else 0 if DIM_RESIZE_METHOD != 2 else None

#out_dimension = 128
word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                              pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
  
print(f'\n---bi_encoder---------------------------')
print(bi_encoder1)
print(word_embedding_model1)
#------------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------------------
# 차원축소
#------------------------------------------------------------------------------------------------------------------------
#  출력 값 차원 축소 지정인 경우, token_embeddings 일때는 sentencebert 라이브러리를 이용하여 dense_model 모델 추가할수 없으므로,
#  사용자정의 dense_model을 정의해서, 가중치와 bias를 bi_encoder모델에 것을 얻어와서 적용해서 차원 죽소함.
# - resize 방식 보다 성능 떨어지지만, 128일때는 더 성능이 좋음
# - weight와 bias 변경 해서 테스트시.klue-sbert-v1-weight-128 가장 성능 좋음(43.80%)
#  => klue-sbert-v1-weight-128.pt=43.80%, kpf-sbert-v1.1-weight-128=42%, 
#     all-mpnet-base-v2-bias-128/paraphrase-multilingual-mpnet-base-v2-bias-128=42.20%, bert-base-multilingual-uncased-bias-128=41.20% 
if DIM_RESIZE_METHOD == 2:
    dense_weight_path = './data/dense_weight/klue-sbert-v1-weight-128.pt'
    bias_weight_path = './data/dense_weight/klue-sbert-v1-bias-128.pt'
    
    #-------------------------------------------------------------------------
    # 처음에는 아래 코드를 활용하여 해당 모델의 128 weight와 bias를 저장해 두어야 함.
    #state_dict = bi_encoder1.state_dict()  # bi_encoder모델이 state_dict 얻어옴
    #print(state_dict.keys())
    #dense_weight = state_dict['2.linear.weight'] # denser 모델에 bi_encoder모델이 linear.weight 적용
    #dense_bias = state_dict['2.linear.bias']     # denser 모델에 bi_encoder모델이 linear.bias 적용
    
    # 처음에  weigth, bias 파일을 저장해 둠.
    #torch.save(dense_weight, dense_weight_path)
    #torch.save(dense_bias, bias_weight_path)
    #-------------------------------------------------------------------------
    # weigth, bias 저장해둔 파일 로딩
    dense_weight = torch.load(dense_weight_path)
    dense_bias = torch.load(bias_weight_path)

    print(f'\n---dense param---------------------------')   
    print('*dense_weight:{}({})'.format(dense_weight.size(), dense_weight_path))
    print(f'*dense_bias:{dense_bias.size()}({bias_weight_path})')
#------------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------------------
# onnx 모델 로딩
#------------------------------------------------------------------------------------------------------------------------
if IS_ONNX_MODEL == True:
    onnx_model_path = "bongsoo/klue-sbert-v1-onnx"#"../../../data11/model/onnx/klue-sbert-v1-onnx" #"bongsoo/klue-sbert-v1-onnx"
    onnx_tokenizer, onnx_model = onnx_model(onnx_model_path)
    print(f'\n---onnx_model---------------------------')
    print(onnx_model)
#------------------------------------------------------------------------------------------------------------------------


---bi_encoder---------------------------
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)
Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 


In [3]:
# 510 토큰 단위로 슬라이딩 윈도우 처리 후 1차원 배열로 출력함
from myutils import sliding_window_tokenizer, get_text_chunks

paragraph = '''
내년 지방자치법 전부 개정안 시행을 앞두고 진정한 지방분권 실현 과제를 모색하기 열린 토론회에서 전문가들은 지방분권 개헌과 함께 실효성 있는 자치입법권을 확보해야 한다고 주장했다.\n대통령소속 자치분권위원회와 대한민국지방신문협의회(대신협)는 10일 오후 2시 제주시리우스호텔 시리우스홀에서 '자치분권 2.0시대 어떻게 맞을 것인가'를 주제로 문재인 정부 출범 4주년 및 지방자치부활 30주년 기념 제주권 대토론회를 개최했다.\n토론회에는 김중석 대신협 회장 겸 대통령 소속 자치분권위원회 자치제도분과위원장, 고영권 제주특별자치도 정무부지사, 김태환 전 제주자치도지사, 김태석 전 제주자치도의회 의장, 이상봉 제주자치도의회 행정자치위원회 위원장, 김용구 제주자치도자치경찰위원회 위원장, 고종석 제주자치도특별자치제도추진단장이 참석했다. 또 이용곤 한라일보 대표이사, 양치석 제민일보 사장 등 대신협 회원사 임원진이 자리했다.\n김중석 위원장은 인사말에서 \"새로운 지방자치법에도 명시되었듯 이제까지 중앙정부로부터 지도·감독을 받아야 했던 수직·종속적 관계를 벗어나 대등·협력 관계로 지방행정 시스템의 틀을 혁신해야 한다\"고 말했다.\n이날 제주권 대토론회는 자치분권 강화를 위한 대안 모색과 진정한 제주특별자치도로 나가기 위한 과제를 도출하는 방식으로 진행됐다.\n김중석 위원장은 '문재인 정부 자치분권 관련 법제의 성과와 의미'라는 발제를 통해 \"주민주권론을 토대로 자치입법권을 강화하는 한편, 중앙과 지방 간 협력 거버넌스를 구축해야 한다\"며 \"또 다가오는 대선 길목에서 후보들로부터 지방분권 개헌과 자치분권 강화를 공약으로 확약 받아야 한다\"고 말했다.\n민기 제주대학교 행정학과 교수는 '자치분권 2.0시대-제주특별자치도의 대응과 과제'란 발제에서 \"특별자치도 출범 이후 4600여건의 중앙행정 권한과 특례가 제주자치도로 이양 또는 신설됐지만 최종적인 입법 형성권은 국회와 중앙정부에 있다\"며 \"현재까지 이양된 권한 중에서 제주자치도의 고유사무에 대해선 조례특례 유형 형식으로 변경해 자 사무에 대한 제주도민의 자기결정권을 강화하는 방향으로 특별자치도의 고도화를 추진해야 한다\"고 주장했다. 대신협 공동취재단
광주상공회의소는 광주시와 공동으로 2021년 '광주형일자리 희망 컨설팅'에 참여할 기업을 오는 21일까지 모집한다.\n광주형일자리 희망 컨설팅은 지역내 광주형일자리 확산을 통해 좋은 일자리 창출과 산업의 경쟁력 강화를 위해 지역 기업이 적정임금, 적정 근로시간, 노사책임경영, 원·하청 관계 개선 등 4대 의제를 도입할 수 있도록 전문가 현장 컨설팅을 지원하는 사업이다.\n올해 지역 기업 20개 사를 대상으로 6월부터 11월까지 5개월여 기간 동안 전문 컨설턴트가 5차례 이상 직접 방문해 무료로 컨설팅을 지원한다.\n각종 행정, 재정적 인센티브와 인증 지원금 혜택을 받는 광주형일자리 기업 인증을 희망하는 업체의 경우 광주형일자리 희망 컨설팅에 반드시 참여해야 한다.\n광주상의 관계자는 \"지역 기업들의 광주형일자리 4대 의제 도입 확대로 지역 내 노사상생 문화가 정착되고, 기업 경쟁력이 제고될 수 있도록 지원을 강화해 나갈 계획이다\"고 밝혔다
KIA타이거즈 2년차 투수 정해영(20)이 올시즌 수호신으로 거듭나고 있다. 묵직한 직구와 예리한 슬라이더, 포크볼을 앞세운 자신감있는 승부로 KIA의 뒷문을 단단히 걸어 잠그고 있다.\n정해영은 지난 9일 대구삼성라이온즈파크에서 열린 삼성과의 원정 경기에서 7-5로 앞선 8회말 1사 1·2루 상황에서 구원 등판해 2점 차 승리를 지켜내며 팀의 3연패 탈출에 기여했다.\n정해영은 1.2이닝 동안 안타 1개를 내줬지만 삼진 2개를 솎아내며 시즌 9세이브째를 따냈다.\n이날 삼성전 최고 하이라이트는 8회말이었다. KIA는 1회초 최형우의 투런홈런과 4회초 황대인의 투런홈런으로 4점을 앞서다 5회말 2실점하며 추격당했다. 7회초 최원준의 1타점 적시타와 김태진의 2타점 적시타로 승기를 잡는 듯 했으나 7회말 3실점하며 7-5로 쫓겼다.\n그리고 8회말 장현식이 1사 후 볼넷과 사구를 내주며 1·2루 위기 상황을 맞자 마무리 정해영을 일찍 마운드에 올렸다.\n정해영의 첫 상대는 1회말 우전안타와 5회말 1타점 내야안타, 7회말 스리런홈런포를 날리며 최고조의 타격감을 선보인 구자욱이었다.\n장타 하나면 동점까지 몰릴 수 있는 상황에서 정해영은 집중력을 발휘했다. 초구로 던진 141㎞ 직구는 파울이 됐다. 2구 슬라이더는 볼이 됐지만 3구를 포크볼로 던져 구자욱의 헛방망이질을 유도했다.\n1볼-2스트라이크의 유리한 상황에서 4구 직구와 5구 슬라이더는 스트라이크존을 통과하지 못하며 풀카운트가 됐다. 구자욱의 끈질긴 승부로 6구 직구와 7구 슬라이더는 파울이 됐다. 정해영은 8구째 132㎞ 슬라이더를 다시 던져 포수 파울 뜬공으로 잡아냈다. 이어 4번타자 오재일을 2루수 땅볼로 처리하며 실점없이 이닝을 마무리지었다.\n9회말에도 마운드에 오른 정해영은 안타 1개를 맞았으나 삼진 2개를 곁들여 무실점 방어에 성공, 팀의 승리를 지켰다.\n정해영의 세이브가 빛나는 이유는 지난해 마무리 전상현의 부재로 올시즌 KIA 불펜진이 크게 흔들리고 있어서다.\n전상현은 지난 시즌 8월 중순 기존 마무리 문경찬이 NC로 트레이드되면서 새 마무리로 나서 2승 2패 15세이브 13홀드, 평균자책점 2.45를 기록하며 올시즌에도 소방수로 기대됐었다.\n하지만 전상현이 스프링캠프 초반 어깨 통증을 호소해 재활군으로 내려간 이후 합류하지 못했다.\n대안으로 꼽혔던 박준표도 부진과 부상이 겹치면서 KIA는 2년 차 투수 정해영을 올시즌 마무리로 활용하고 있다.\n정해영은 9일까지 23경기에서 4승3패 9세이브 평균자책점 2.96을 기록하고 있다.\n4월 등판한 11경기 중 1경기에서만 1실점하고 나머지 10경기에서는 무실점 방어하며 평균자책점 0.69로 철벽 마무리를 자랑했다. 5월엔 9경기에 등판해 3경기에서 실점하며 블론세이브를 기록했지만 6월 들어 3경기에 출전해 모두 무실점 방어를 펼쳤다.\n정해영의 마무리 성공 비결은 직구 평균 구속 향상과 자신감을 꼽을 수 있다. 정해영의 지난시즌 직구 평균 구속이 141㎞ 정도였는데 올시즌 3~4㎞ 향상됐고, 공을 앞으로 끌고 나오는 익스텐션 동작이 리그 상위권 수준이어서 타자들이 상대적으로 빠르고 묵직하게 느껴진다는 게 KIA 코칭스태프의 설명이다.\n여기에 자신의 공을 믿고 과감하게 정면 승부하는 강심장을 가졌기에 앞으로 특급 마무리로 성장 가능성이 높아 보인다.
무안군은 지난해 5월1일 건축물관리법이 시행됨에 따라 모든 건축물 해체(철거) 시 사전에 건축물 해체 허가를 받거나 신고를 해야 한다고 10일 밝혔다.\n건축물 해체 신고대상 건축물은 △주요 구조부 해체 없이 건축물의 일부를 해체할 경우 △연면적 500㎡ 미만, 높이 12m 미만, 지상층과 지하층을 포함한 3개 층 이하 등 모든 사항을 만족하는 건축물의 전체를 해체할 경우 △관리지역·농림지역·자연환경보전지역 내 높이 12m 미만인 건축물을 해체할 경우이다.\n이외의 건축물의 해체는 허가대상이며 해체 허가(신고) 절차를 이행하지 않을 경우 최대 500만원의 과태료가 부과될 수 있다.\n해체 허가 대상 건축물의 경우 건축물 해체 허가 신청 시 건축사, 기술사, 안전진단전문기관 등 전문가에게 검토받은 해체공사계획서를 제출해야 하며 안전한 현장관리를 위해 해체공사 감리자를 지정해야 한다.\n해체 허가(신고)를 받은 건축물을 전면 해체해 공사가 완료된 경우에는 건축물 해체공사 완료 신고를 하면 건축물을 멸실 신고 한 것으로 본다.\n멸실 신고 절차를 이행하지 않을 경우도 최대 100만원의 과태료가 부과될 수 있으므로 주의해야 한다.\n무안군 관계자는 \"건축물 관리자는 해체허가 신고 및 멸실 신고 절차를 준수해 과태료가 부과되는 등의 불이익 처분을 받는 일이 없도록 해야한다\"며 \"건축물 해체 시 안전사고를 예방하고자 마련된 제도이므로 적극 협조해 주길 바란다\"고 말했다
장흥군(군수 정종순)은 민선7기 공약의 실천계획 변경에 군민들이 직접 참여하고 결정하는 민주적 절차인 '주민배심원제'를 운영한다고 10일 밝혔다.\n군은 지난 8일 장흥통합의학컨벤션센터에서 주민배심원단, 군 관계자, 한국매니페스토실천본부 관계자 등 30여 명이 참여한 가운데 '민선7기 공약이행 현황 점검을 위한 주민배심원 3차 회의'를 개최했다.\n'주민배심원제'는 공약 실천계획 설정에 있어 공약의 당사자인 지역 주민이 중심이 돼 참여함으로써 공약 내용 조정에 대해 직접 심의하고 개선안을 제안할 수 있는 제도이다. 객관성 확보를 위해 전문기관인 한국매니페스토실천본부에서 진행했다.\n주민배심원은 대표성 확보를 위해 주민등록상 만 18세 이상의 장흥군 거주 주민들 중에서 성별·나이·지역별 인구 비례에 따른 무작위 추첨 방식(유선전화 ARS)으로 350명의 배심원 후보자를 1차 선발했다. 이 중 주민배심원 참여 의사를 밝힌 주민을 대상으로 전화 면접을 거쳐 최종 배심원 35명을 선발했다.\n올해 주민배심원단은 총 3차 회의를 진행했다. 지난달 11일 1차 회의에서는 위촉장 수여, 분임구성, 기초교육을 진행했으며, 지난달 25일 2차 회의에서는 분임별 공약 안건 설명 및 질의응답, 분임별 자유토의 등이 활발히 이뤄졌다.\n3차 회의에서 주민배심원단은 △정남진장흥노인복지재단 기금 50억원 조성 및 노인복지사업 확대 △예양 숲공원 조성 △정남진 종합 스포츠타운 조성 △중앙로 주차난 해소를 위한 소형 주차장 설치 등 총 14개 공약에 대해 분임별 토의를 거쳐 공약 조정의 적정 여부를 심의하고 개선방안을 제안했다.\n정종순 장흥군수는 \"주민들과 직접 소통하는 행정으로 변화시켜 나가기 위해 군민과의 약속인 공약을 더욱 성실히 이행해 신뢰도와 만족도를 높여 나가겠다\"고 전했다
삼성전자가 세척부터 건조·살균까지 업그레이드해 한국 식문화에 최적화된 '비스포크 식기세척기' 신제품을 출시했다고 10일 밝혔다.\n지난해 6월 선보인 삼성 비스포크 식기세척기는 4단 입체 물살을 통한 강력하고 빈틈없는 세척과 전면 도어 패널을 교체할 수 있는 비스포크 디자인으로 호평을 받아왔다.\n이번에 새롭게 출시된 비스포크 식기세척기는 '360도 제트샷'과 '열풍건조' 기능을 더해 세척 성능뿐만 아니라 건조와 살균까지 한층 강화한 것이 특징이다.\n'360도 제트샷'은 뜨거운 물살을 집중적으로 쏘아주는 기능으로, 대형 식기가 많이 놓이는 하단 후면에 적용해 음식물이 눌어붙은 냄비나 프라이팬의 때 등을 효과적으로 제거한다.\n비스포크 식기세척기는 △'360도 제트샷' △식기에 눌어붙은 밥풀이나 양념을 닦기 쉽게 만들어 주는 '스팀 불림' △사각지대 없이 꼼꼼하게 세척해 주는 '4단 입체 물살' 기능을 통해 애벌 세척을 하지 않아도 깨끗하게 설거지를 해 주는 '3단계 클린세척'을 구현했다.\n이번 신제품은 '열풍건조' 기능을 도입해 5단계 건조 시스템을 구현하고 식기를 더욱 깔끔하고 효율적으로 건조해줄 뿐만 아니라 위생 기능도 대폭 강화했다.\n'열풍건조'는 내부의 습한 공기를 흡입한 후 가열해 만든 뜨거운 바람으로 식기의 잔류 물방울까지 말끔하게 건조해 주는 기능이다.\n'젖병살균' 코스의 경우 세제 없이도 젖병이나 유아 식기에 있는 대장균, 살모넬라균, 리스테리아균 등의 유해 세균과 로타, 노로, A형간염 등의 바이러스를 99.999% 제거한다.\n'통살균' 코스를 활용하면 세제 없이도 고온직수와 열풍으로 식기세척기 내부를 살균해 한층 위생적으로 사용할 수 있다.\n이 밖에도 식기세척기 손잡이에는 항균 소재를 적용해 황색포도상구균, 대장균 등 각종 세균 증식을 억제하도록 했다.\n설거지 양이 적을 때는 '상단 급속' 코스를 사용하면 세척부터 건조까지 29분 만에 끝낼 수 있다. 제품 중단에 볼베어링 롤러를 적용해 식기로 인해 무거워진 바구니를 손쉽게 넣고 뺄 수 있도록 했고 일부 모델에는 다양한 크기의 와인잔을 놓을 수 있는 '접이식 와인잔 랙'도 추가됐다.\n비스포크 식기세척기 라인업은 프리스탠딩·빌트인 겸용과 빌트인 전용 제품으로 구성된다. 글램, 새틴, 코타, 페닉스 등 총 4가지 소재, 14가지 색상으로 출시돼 설치 공간의 구조와 인테리어에 맞춰 선택할 수 있다.\n특히 빌트인 전용 제품은 걸레받이 높이가 낮아지는 최근 주방 트렌드를 반영해 가구장에 딱 맞게 설치할 수 있는 구조로 설계됐으며 도어 패널은 필요 시 교체가 가능하다.\n비스포크 식기세척기의 출고가는 제품 타입과 색상에 따라 144~184만원이다.\n삼성전자는 비스포크 식기세척기 신제품 출시를 맞아 소비자 체험단을 모집하고 추첨을 통해 비스포크 식기세척기와 비스포크 인덕션을 직접 사용해 볼 수 있는 기회를 제공한다.\n행사에 참여하고자 하는 고객은 삼성닷컴 홈페이지에서 11일부터 내달 8일까지 응모하면 된다.\n한편 삼성전자는 식기세척기 신제품을 마지막으로 지난 3월 전격 공개한 냉장고, 정수기, 인덕션, 에어컨, 청소기, 세탁기, 건조기, 에어드레서 등으로 이뤄진 '비스포크 홈' 20종 제품을 모두 출시했다.
'''

if IS_ONNX_MODEL == True:
    tokenizer = onnx_tokenizer.tokenizer
else:
    tokenizer = word_embedding_model1.tokenizer
    
# result = sliding_window_tokenizer(tokenizer = tokenizer, paragraph=paragraph, window_size=WINDOW_SIZE, sliding_size=SLIDING_SIZE)
result = get_text_chunks(tokenizer = tokenizer, paragraph=paragraph, chunk_token_size=WINDOW_SIZE)
print(len(result))
for i in range(len(result)):
    print(result[i])
    print()


Token indices sequence length is longer than the specified maximum sequence length for this model (3278 > 512). Running this sequence through the model will result in indexing errors


15
내년 지방자치법 전부 개정안 시행을 앞두고 진정한 지방분권 실현 과제를 모색하기 열린 토론회에서 전문가들은 지방분권 개헌과 함께 실효성 있는 자치입법권을 확보해야 한다고 주장했다. 대통령소속 자치분권위원회와 대한민국지방신문협의회 ( 대신협 ) 는 10일 오후 2시 제주시리우스호텔 시리우스홀에서'자치분권 2. 0시대 어떻게 맞을 것인가'를 주제로 문재인 정부 출범 4주년 및 지방자치부활 30주년 기념 제주권 대토론회를 개최했다. 토론회에는 김중석 대신협 회장 겸 대통령 소속 자치분권위원회 자치제도분과위원장, 고영권 제주특별자치도 정무부지사, 김태환 전 제주자치도지사, 김태석 전 제주자치도의회 의장, 이상봉 제주자치도의회 행정자치위원회 위원장, 김용구 제주자치도자치경찰위원회 위원장, 고종석 제주자치도특별자치제도추진단장이 참석했다. 또 이용곤 한라일보 대표이사, 양치석 제민일보 사장 등 대신협 회원사 임원진이 자리했다.

위원장은 인사말에서 " 새로운 지방자치법에도 명시되었듯 이제까지 중앙정부로부터 지도 · 감독을 받아야 했던 수직 · 종속적 관계를 벗어나 대등 · 협력 관계로 지방행정 시스템의 틀을 혁신해야 한다 " 고 말했다. 이날 제주권 대토론회는 자치분권 강화를 위한 대안 모색과 진정한 제주특별자치도로 나가기 위한 과제를 도출하는 방식으로 진행됐다. 김중석 위원장은'문재인 정부 자치분권 관련 법제의 성과와 의미'라는 발제를 통해 " 주민주권론을 토대로 자치입법권을 강화하는 한편, 중앙과 지방 간 협력 거버넌스를 구축해야 한다 " 며 " 또 다가오는 대선 길목에서 후보들로부터 지방분권 개헌과 자치분권 강화를 공약으로 확약 받아야 한다 " 고 말했다. 민기 제주대학교 행정학과 교수는'자치분권 2.

- 제주특별자치도의 대응과 과제'란 발제에서 " 특별자치도 출범 이후 4600여건의 중앙행정 권한과 특례가 제주자치도로 이양 또는 신설됐지만 최종적인 입법 형성권은 국회와 중앙정부에 있다 " 며 " 현재까지 이양된 권한 중에서 제주자치도의 고유사무에 대해선 조례

In [4]:
#-------------------------------------------------------------------------------------------------------
# 2. JSON 파일 로딩 후 df 만듬
# => KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------
from myutils import read_korquad_v1_json, read_aihub_qua_json

# list 데이터들을 dataframe으로 만듬
'''
contexts = [
        [1, '독도 독도 독도 해역 헬기 추락사고가 발생한 지 열하루가 지났지만 실종자 추가 발견 소식은 들려오지 않고 있다. 헬기 동체 잔해물과 부유물 등은 발견되고 있지만 정작 실종자들은 발견하지 못해 수색이 장기화될 것이라는 우려가 현실이 될 조짐을 보여 실종자 가족들의 애를 태우고 있다.범정부현장수습지원단(지원단)은 10일 오전 10시 브리핑에서 이날 오전까지 독도해역 수색 결과 4점의 부유물을 추가 발견, 인양했다고 밝혔다'], 
        [11,'대구시는 11일 내년 정부 예산 3조1330억원을 확보해 전년보다 2%(611억원) 늘어났다고 밝혔다.그러나 올해 국비 증액 규모(1817억원)와 비교해 절반 수준에 그치며, 물산업클러스터 r&d 사업(200억), 국립청소년진로직업체험 수련원 건립 등은 한푼도 반영되지 않았다.'],
        [21,'자유한국당 (이름) 의원이 "공수처는 정권 유지를 위한 수단으로, 공수처가 있었으면 조국 수사를 못했을 것"이라고 주장했다.최 의원은 18일 대구 호텔수성에서 열린 대구 경북중견언론인모임 <아시아포럼21> 토론회에서 "검찰은 정권 말기가 되면 정권에 칼을 들이댔다. 공수처는 검찰의 칼끝을 회피하기 위한 것"이라며 이같이 말했다'],
        [2,'27일 오전 천연기념물(201-2호)이며 멸종위기 2급인 큰고니 떼가 경북 포항시 북구 흥해읍 샛강에 날아들었다.이날 관측된 큰고니는 어미 8마리와 새끼 3마리다.앞서 지난 19일 큰고니 8마리가 올들어 처음으로 관측됐다.큰고니들은 포항시 흥해읍 샛강에서 수초 등을 먹고 휴식한 후 대구 안심습지와 낙동강 하구 등지로 날아갈 것으로 보인다'],
        [3,'3일 대구와 경북지역은 구름이 많겠으며, 동해안과 북부 내륙에는 비가 내릴 것으로 예상된다.대구기상청에 따르면 중국 북동지방에서 남하하는 고기압의 가장자리에 들어 구름이 많겠으며, 경북 북부 내륙은 북쪽을 지나는 약한 기압골의 영향을 받아 오후부터 비가 약하게 내리거나 빗방울이 떨어지겠다.'],
        [31,'각급 정부기관의 통신망을 관리할 국가정보자원관리원 대구센터가 31일 대구 동구 도학동에서 착공했다.4312억원을 투입해 2021년 8월 준공 예정인 대구센터는 86개 기관의 서비와 장비를 운용하게 된다.정부통합전산센터에서 명칭이 변경된 국가정보자원관리원은 현재 대전본원과 광주센터를 운영 중이다.대구센터는 급변하는 행정환경과 4차 산업혁명 시대에 맞춰 클라우드, 빅데이터 등 신기술이 접목된 지능형 전산센터로 구축된다'],
        [4,'26일 오후 11시44분쯤 대구 달서구 신당동의 한 아파트에서 원인 모를 불이 나 주민 1명이 다치고 수십명이 대피했다.27일 소방당국에 따르면 신고를 받고 소방차 24대와 소방대원 74명이 출동해 20분만에 불길을 잡았다.이 불로 아파트 2층에 사는 a씨(53)가 온 몸에 화상을 입어 병원으로 이송됐으며, 연기에 놀란 주민 50여명이 대피하는 소동을 빚었다.']
       ]
df_contexts = pd.DataFrame(contexts, columns=['contextid','context'])
print(df_contexts['context'].values)
'''

# KorQuAD_v1.0 혹은 aihub 뉴스 기사 기계독해 데이터 QuA 파일을 불러옴.
jsonfile = './data/KorQuAD_v1.0_dev.json' # VL_unanswerable.json # VL_text_entailment.json # VL_span_inference.json # KorQuAD_v1.0_dev.json
contexts, questions, answers, contextids, qcontextids = read_korquad_v1_json(jsonfile) # read_aihub_qua_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [8]:
#-------------------------------------------------------------------------------------------------------
# 3. 문장(문서)들을 chunk(청크: 큰 덩어리)로 분리
# => 512 단위로 서로 겹치게 청크 단위로 분리함.
#-------------------------------------------------------------------------------------------------------
from tqdm.notebook import tqdm
from myutils import sliding_window_tokenizer

#text = ["해외에서 데이터 무제한?? 뿐 아니라 음성과 문자 요금을 할인 받을 수 있는 ++요금제가 나왔다. 날씨는 좋다. lg유플러스는 중국, 일본, 홍콩, 싱가포르, 필리핀 등 아시아 8국을 대상으로~~ 무제한 데이터와 음성, 문자를 할인해주는 '스마트 로밍 요금제' 2종을 오는 28일부터 판매한다고 27일 밝혔다. . . 우선 '스마트 로밍음성'은 하루 기본료가 2000원으로 음성발신은 1분당 1000원이며, 문자메시지(sms)와 멀티미디어 문자메시지(mms)는 1건당 150원이다. 종전 500원에서 350원 인하했다. . . '스마트 로밍패키지'는 여기에 데이터 무제한 로밍서비스가 더해져 하루 기본료가 1만1000원이다. 기본료는 로밍 기간에 상관없이 사용한 당일에만 청구된다. . . 스마트 로밍 요금제는 일단 오는 7월 말까지 프로모션 형태로 제공하고 이후 정식 요금제로 추진한다는 계획이다. . . 아울러 해외에서 무제한 데이터 로밍 서비스와 데이터로밍 차단을 신청·해지할 수 있는 로밍 전용 모바일 홈페이지((이메일))도 운영한다. . . (이름) lg유플러스 글로벌로밍팀장은 '아시아 출(이름) 여행을 계획하고 있는 고객들이 저렴한 비용으로 안심하고 로밍 서비스를 사용할 수 있도록 이번 프로모션 요금제를 출시했다'며 '지속적으로 해외 로밍 이용 고객들의 편의성 증대를 위한상품을 준비할 예정'이라고 말했다. . . . . . . . # # #. . ■ 사진설명. . lg유플러스(부회장 (이름) / (이메일) )가 4월 28일부터 중국, 일본 등 아시아 8개국*을 대상으로 데이터와 음성 문자까지 할인해서 제공하는 스마트 로밍요금제(스마트 로밍음성/스마트 로밍패키지) 2종을 출시했다."]
#text =["오늘은 날씨가 ^^~~~~좋다. 내일은 비가 오고 춥겠다고 한다^^. 걱정이다. 오늘만큼만 매일 날씨가 좋으면 좋겠다.~~~~"]
sub_contexts = []
sentences = []

contexts = df_contexts['context'].values.tolist()

start = time.time()

# 슬라이딩 윈도우 처리 후 sub_contexts 리스트에 2차원 배열로 문단들을 담음.
if IS_ONNX_MODEL == True:
    tokenizer = onnx_tokenizer
    print(tokenizer)
else:
    tokenizer = word_embedding_model1.tokenizer
    
for idx, context in enumerate(tqdm(contexts)):
    #result = sliding_window_tokenizer(tokenizer = tokenizer, paragraph=context, window_size=WINDOW_SIZE, sliding_size=SLIDING_SIZE)
    result = get_text_chunks(tokenizer = tokenizer, paragraph=context, chunk_token_size=WINDOW_SIZE)
    sub_contexts.append(result)
    
logger.info(f'*문장 분리 시간 : {time.time()-start:.4f}')
print(len(sub_contexts[0]))
print(sub_contexts[0])

'''
#contexts = text
start = time.time()
sub_contexts = split_sentences(paragraphs=contexts, sentences_split_num=10, paragraphs_num=999, debug=False)
logger.info(f'*문장 분리 시간 : {time.time()-start:.4f}')

print(sub_contexts[0])
'''

  0%|          | 0/964 [00:00<?, ?it/s]

2023-03-27 14:23:12,887 - MRR-sliding - INFO - *문장 분리 시간 : 1.7943
[Kss]: *문장 분리 시간 : 1.7943


1
['1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.']


"\n#contexts = text\nstart = time.time()\nsub_contexts = split_sentences(paragraphs=contexts, sentences_split_num=10, paragraphs_num=999, debug=False)\nlogger.info(f'*문장 분리 시간 : {time.time()-start:.4f}')\n\nprint(sub_contexts[0])\n"

In [14]:
from myutils import embed_text, onnx_embed_text

# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragrphs:list):
    if IS_ONNX_MODEL == True:
        if IS_EMBED_DIVIDE == True: # 한 문단에 대한 40개 문장들을 토큰단위로 쪼개서 임베딩 처리함  
            #----------------------------------------------------
            # 한 문단에 대한 문장들의 토큰을 ?개씩 나누고 비교.
            # - 한 문단에 대한 문장들에 대해 [tensor(250,768), tensor(243,768), tensor(111,768),..] tensor 리스트 타입으로 벡터 생성됨.
            #----------------------------------------------------
            embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs) 
            
        else: # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
            embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs, token_embeddings=False)
    else:
        if IS_EMBED_DIVIDE == True: # 한 문단에 대한 40개 문장들을 토큰단위로 쪼개서 임베딩 처리함  
            #----------------------------------------------------
            # 한 문단에 대한 문장들의 토큰을 ?개씩 나누고 비교.
            # - 한 문단에 대한 문장들에 대해 [tensor(250,768), tensor(243,768), tensor(111,768),..] tensor 리스트 타입으로 벡터 생성됨.
            #----------------------------------------------------
            embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, token_embeddings=True, return_tensor=False)
        else: # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
            embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, return_tensor=False)  
    
    return embeddings
   

In [15]:
#-------------------------------------------------------------------------------------
# 4. 문단(문서)별 문장들의 토큰 평균들에 대해 Faiss 임베딩 생성
# - 각 문단의 문장들에 대해 토큰들의 평균값을 구하고, Faiss에 임베딩 추가함.
#-------------------------------------------------------------------------------------
import torch
from myutils import embed_text, dense_model, fassi_index, divide_arr_avg, divide_arr_avg_exten

EMBED_CONTEXTS = sub_contexts#sub_contexts #sub_vocab_contexts # 임베딩을 구해 인덱스에 추가할 문장들 1차원 리스트 (예:['오늘은 좋다','날씨가 흐리다','제주도 날씨',...])
faissindexlist = []

logger.info(f'*faiss 인덱싱 시작 => 방식: 토큰 분리:{IS_EMBED_DIVIDE}({EMBED_DIVIDE_LEN})/{faiss_index_method}(0=코사인, 1=유클리드)/차원축소:{DIM_RESIZE_METHOD}({DIM_RESIZE_LEN})/ONNX:{IS_ONNX_MODEL}')
    
start = time.time()
embed_context = []
embed_len = []
# EMBED_CONTEXTS 2차원 리스트 형태 : 예시 [['날씨가좋다','오후에는 비가온다','내일은 춥다'],['사과는맛있다','배도 맛있다'],[],[],...]
for idx, embed_context in enumerate(tqdm(EMBED_CONTEXTS)):
     
    # 임베딩 처리
    if IS_EMBED_DIVIDE == False:
        embeddings = embedding(embed_context)
    else:
        token_embeds = embedding(embed_context)
    
    #------------------------------------------------------------------------------------------------------------------------ 
    # 토큰 분할인 경우 처리 start=> 
    if IS_EMBED_DIVIDE == True:
        token_embed_arr_list = []
        tcount = 0
        # tensor(250,768) 한문장 토큰 임베딩 얻어와서, 각 ?개 토큰씩 평균을 구함.
        for token_embed in token_embeds:
            token_embed = token_embed[1:-1] # 맨앞에 [CLS]와 맨뒤에 [SEP] 토큰은 뺌
            
            if tcount >= MAX_TOKEN_LEN: 
                break
                '''
           # Dense 방식으로 차원 축소 => 평균 구하기 전에 차원 축소
            if DIM_RESIZE_METHOD == 2:
                token_embed = dense_model(embed_tensor=token_embed, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias)
                token_embed_arrs = token_embed.detach().numpy().astype('float32')
            else:
                # tensor를 arrary로 변경
                token_embed_arrs = token_embed.cpu().numpy().astype('float32')
                '''
            # tensor를 arrary로 변경
            token_embed_arrs = token_embed.cpu().numpy().astype('float32')
            
             # 7,8,10 씩 자르면서 문장 토큰 평균을 구함
            token_embed_divide_arrs = divide_arr_avg_exten(embed_arr=token_embed_arrs, divide_arrs=EMBED_DIVIDE_LEN, debug=False)
            #arrs = token_embed_divide_arrs
   
                    
            # Dense 방식으로 차원 축소 => 평균 구한후 차원 축소하는 방식이 0.6% 정도 성능 좋음
            if DIM_RESIZE_METHOD == 2:
                tmp1 = torch.Tensor(token_embed_divide_arrs)
                #tmp1 = torch.from_numpy(token_embed_divide_arrs)
                tmp2 = dense_model(embed_tensor=tmp1, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias, debug=False, idd=idx)                
                arrs = tmp2.detach().numpy().astype('float32')
      
            else:  
                arrs = token_embed_divide_arrs

            # 평균 구한 토큰들을 token_embed_arr_list 리스트에 담아둠.(50보다 크면 50개만 담음)      
            
            for idx, arr in enumerate(arrs):                  
                # Resize 방식으로 차원 축소(384로 줄일대 -2% 성능 저하 발생)
                if DIM_RESIZE_METHOD == 1:
                    darr = np.resize(arr, (DIM_RESIZE_LEN,))
                else:
                    darr = arr
                
                token_embed_arr_list.append(darr)
                tcount +=1
                if tcount >=MAX_TOKEN_LEN:
                    break
                
        embeddings = np.array(token_embed_arr_list)
        # 토큰 분할인 경우 처리 End<== 
        #------------------------------------------------------------------------------------------------------------------------  
        
    embed_len.append(len(embeddings))
            
    # Faiss index 생성하고 추가 
    index = fassi_index(embeddings=embeddings, method=faiss_index_method)
    faissindexlist.append(index)
    
logger.info(f'*임베딩 시간 : {time.time()-start:.4f}')
logger.info(f'*임베딩 shape : {embeddings.shape}')

ecount = 0
for elen in embed_len:
    if elen >= MAX_TOKEN_LEN:
        ecount+=1
        
logger.info(f'*인덱스별 임베딩 수 : {embed_len}/*최대값:{max(embed_len)}/*{MAX_TOKEN_LEN} <= 계수:{ecount}')

2023-03-27 14:26:12,209 - MRR-sliding - INFO - *faiss 인덱싱 시작 => 방식: 토큰 분리:False([5, 7, 9])/0(0=코사인, 1=유클리드)/차원축소:0(128)/ONNX:False
[Kss]: *faiss 인덱싱 시작 => 방식: 토큰 분리:False([5, 7, 9])/0(0=코사인, 1=유클리드)/차원축소:0(128)/ONNX:False


  0%|          | 0/964 [00:00<?, ?it/s]

2023-03-27 14:29:33,689 - MRR-sliding - INFO - *임베딩 시간 : 201.4768
[Kss]: *임베딩 시간 : 201.4768
2023-03-27 14:29:33,691 - MRR-sliding - INFO - *임베딩 shape : (1, 768)
[Kss]: *임베딩 shape : (1, 768)
2023-03-27 14:29:33,693 - MRR-sliding - INFO - *인덱스별 임베딩 수 : [1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 3, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 3, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 1, 3, 2, 2, 2, 6, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 3, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 5, 6, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1

In [16]:
#-------------------------------------------------------------------------------------
# 쿼리문장 샘플링
#user_querys = ["독도에서 사고가 나서 실종자가 발생했다.", "오늘 날씨가 흐리고 비가 오겠다."]
#-------------------------------------------------------------------------------------
from myutils import df_sampling

# 쿼리 샘플링함.
if query_num == 0:   # query_num = 0 이면 모든 쿼리(5533개)
    user_querys = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 랜덤하게 샘플링하여 쿼리 목록을 만듬.
    df_questions = df_sampling(df=df_questions, num=query_num, seed=seed)
    user_querys = df_questions['question'].values.tolist()
  
print(f'Query-----------------------------------------------------')
print(user_querys[0:3])

Query-----------------------------------------------------
['첫 번째 문법조약은 몇 세기에 쓰여졌나?', '노무현 정부 당시 미국의 국무장관은?', '바샤르 알아사드가 집권한 후 시리아의 인권상태를 개선시키는 데 실패했다고 주장한 기관은 어디인가?']


In [17]:
#-------------------------------------------------------------------------------------
# 5.쿼리 후 단어 MAX 스코어 합 구함
# => 쿼리 문장을 각 토큰별 임베딩값을 구하고, 이후 Faiss 문장 임베딩과 비교하여 가장 유사한 문장을 찾음
# => 3,000 문장 GPU 환경에서 약 4분 걸림.
#-------------------------------------------------------------------------------------

if IS_QUERY_EMBED_DIVIDE == True:
    
    from myutils import df_sampling, sum_of_array_2d, index_of_list,split_sentence_list, clean_text

    bi_predictions_list = []

    for i, user_query in tqdm(enumerate(user_querys)):

        if i < 5:
            print(user_query)

        # 쿼리별 토큰 벡터를 구함 
        vectors_tmp = embed_text(model=bi_encoder1, paragraphs=[user_query], token_embeddings=True, return_tensor=False)
        vectors_tmp2 = vectors_tmp[0].cpu().numpy().astype('float32') # tensor 토큰 벡터를 np.array로 변경
        vectors_tmp2 = vectors_tmp2[1:-1] # 맨앞에 [CLS]와 맨뒤에 [SEP] 토큰은 뺌
        
        vectors = divide_arr_avg(embed_arr=vectors_tmp2, divide_len=QUERY_DIVIDE_LEN) # QUERY_DIVIDE_LEN 만큼 자르면서 쿼리 토큰 평균을 구함

        # *cosine유사도 구할때는 반드시 normalize 처리함.
        if faiss_index_method == 0:
            faiss.normalize_L2(vectors)          

        max_values_list = []

        # 기존 contexts에 대한 faiss 문장 인덱스를 불러오면서, 단어 별루 쿼리 검색 해서 스코어를 구하고, 구한 단어별 스코어를 모두 더해서 총합을 구함.
        for count, index in enumerate(faissindexlist):

            # 각 faiss 문장 인덱스에서 문장단어 쿼리 검색 함 (distance, idx= numpy.array 타입임)
            distance, idx = index.search(np.array(vectors).astype("float32"), k=3) # 단어별 검색해서 3개 스코어를 얻어옴.

            # 예) distance = np.array([[2,3,4],[4,3,2]])
            # faiss_index_method=0 => cosine유사도 구할때는 max 값을 찾음, 1=>유클리드 거리 로 할때는 min 값을 찾음
            sum_value = sum_of_array_2d(array=distance, bmin=faiss_index_method)

            # 각 문장별 스코어 최대값을 저장해 둠.
            max_values_list.append(sum_value)

        # 문장별 스코어 최대값 리스트에서 최대값을 갖는 항목 search_k개 index만 출력 함.
        # faiss_index_method=0 => cosine유사도 구할때는 max 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [3,2,4])
        # faiss_index_method=1 => 유클리드 거리 로 할때는 min 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [1,0,4])
        indices = index_of_list(listdata=max_values_list, k=search_k, bmin=faiss_index_method)  

        # 예측검색결과 contextid값들을 리스트로 만듬.
        tmp_bi_predictions_list = []
        for indice in indices:
            tmp_bi_predictions_list.append(df_contexts["contextid"][indice])

        # 2D 예측검색결과 리스트에 추가 
        bi_predictions_list.append(tmp_bi_predictions_list)

    print(bi_predictions_list[0:3])

In [18]:
#-------------------------------------------------------------------------------------
# 5. 쿼리 후 가장 유사한 문장(문서)의 sub 문장 평균 구함
# => 쿼리 문장에 임베딩값을 구하고, 이후 Faiss sub 문장 임베딩과 비교하여 가장 유사한 문장 avg_num개 를 찾고 평균을 구함
#    이후 평균값이 가장 큰 search_k 갯수 만 쿼리에 대한 예측 결과 리스트로 만듬
#-------------------------------------------------------------------------------------

if IS_QUERY_EMBED_DIVIDE == False:
    
    from myutils import df_sampling, sum_of_array_2d, index_of_list,split_sentence_list, clean_text

    bi_predictions_list = []

    # 임베딩 처리
    if IS_EMBED_DIVIDE == False:
        embed_querys = embedding(user_querys)
    else:
        token_query_embeds = embedding(user_querys)

    if IS_EMBED_DIVIDE == True:
        token_query_embed_arr_list = []
        # 쿼리 문장들의 토큰들의 평균을 구함.
        for token_query_embed in token_query_embeds:
            token_query_embed = token_query_embed[1:-1] # 맨앞에 [CLS]와 맨뒤에 [SEP] 토큰은 뺌

            '''
            # Dense 방식으로 차원 축소 => 평균 구하기 전에 차원 축소
            if DIM_RESIZE_METHOD == 2:
                token_query_embed = dense_model(embed_tensor=token_query_embed, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias)
                tmp = token_query_embed.detach().numpy().astype('float32')
            else:
                tmp = token_query_embed.cpu().numpy().astype('float32')
                '''
            tmp=token_query_embed.cpu().numpy().astype('float32')
            tmp=tmp.mean(axis=0) #평균 구함(768,) 1차원배열로 출력됨

            # Resize 방식으로 차원 축소(384로 줄일대 -2% 성능 저하 발생)
            if DIM_RESIZE_METHOD == 1:
                tmp = np.resize(tmp, (DIM_RESIZE_LEN,))
             # Dense 방식으로 차원 축소=> 평균 구하기 전에 차원 축소하는것이 성능 +0.6더 좋음
            elif DIM_RESIZE_METHOD == 2:
                tmp1 = torch.Tensor([tmp]) # 1차원 배열을 -> 2차둰 텐서(1,768)로 변환
                tmp2 = dense_model(embed_tensor=tmp1, out_f=DIM_RESIZE_LEN, weight=dense_weight, bias=dense_bias)
                tmp = tmp2.detach().numpy().astype('float32').ravel(order='C') # 1차원 배열로 변경(128,)            

            token_query_embed_arr_list.append(tmp)

        embed_querys = np.array(token_query_embed_arr_list)  # 리스트를 배열로 변환  
    #------------------------------------------------------------------


    if faiss_index_method == 0:
        faiss.normalize_L2(embed_querys)          # *cosine유사도 구할때는 반드시 normalize 처리함.

    for embed_query in tqdm(embed_querys):
        embed_query = [embed_query]

        max_values_list = []
        for count, index in enumerate(faissindexlist):
            distance, idx = index.search(np.array(embed_query).astype("float32"), k=avg_num) # avg_num 계수 만큼 유사한 sub문장을 찾음
            avg_distance = distance.mean(axis=1) # 검색된 sub 문장들에 대해 평균을 구함
            max_values_list.append(avg_distance[0])

        # 문장별 스코어 최대값 리스트에서 최대값을 갖는 항목 search_k개 index만 출력 함.
        # faiss_index_method=0 => cosine유사도 구할때는 max 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [3,2,4])
        # faiss_index_method=1 => 유클리드 거리 로 할때는 min 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [1,0,4])
        indices = index_of_list(listdata=max_values_list, k=search_k, bmin=faiss_index_method)  

        # 예측검색결과 contextid값들을 리스트로 만듬.
        tmp_bi_predictions_list = []
        for indice in indices:
            tmp_bi_predictions_list.append(df_contexts["contextid"][indice])
            #print(f'*인덱스:{indice}/contextid:{df_contexts["contextid"][indice]}-----------------------------------------------------------')
            #print('*총합/평균 : {:.4f}/{:.4f}'.format(max_values_list[indice], float(max_values_list[indice]/len(mecab_query))))
            #print(df_contexts['context'][indice])
            #print(mecab_contexts[indice])

        # 2D 예측검색결과 리스트에 추가 
        bi_predictions_list.append(tmp_bi_predictions_list)

    print(bi_predictions_list[0:3])


  0%|          | 0/500 [00:00<?, ?it/s]

[[10217, 10359, 10738, 10658, 10242], [10026, 10454, 10377, 10465, 10872], [10312, 10310, 10311, 10072, 10758]]


In [20]:
#--------------------------------------------------------------------------------------------------
# 6. MRR 계산
# => 정답 리스트[2,3,1,4] 과 예측검색리스트[[1,2,5,1],[3,4,2,1],[6,5,4,1], [2,3,4,1]]를 입력하여 MRR 스코어 구함
##--------------------------------------------------------------------------------------------------
from myutils import mean_reciprocal_rank

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
#print(f'gtlen:{len(ground_truths_list)}')
#print(ground_truths_list[0:9])

logger.info(f'--------------------------------------------------------------------------')
logger.info('json_file:{}'.format(jsonfile))
logger.info(f'faiss 인덱싱 방식: max 토큰:{MAX_TOKEN_LEN}/{faiss_index_method}(0=코사인 유사도, 1=유클리드 거리)/차원축소:{DIM_RESIZE_METHOD}({DIM_RESIZE_LEN})/ONNX:{IS_ONNX_MODEL}')
#logger.info(f'슬라이딩 윈도우: window_size:{WINDOW_SIZE}/sliding_size:{SLIDING_SIZE}')

logger.info('*avg_num:{}/search_k:{}/query_num:{}/out_dimension:{}'.format(avg_num, search_k, query_num, out_dimension))

# MRR 계산
bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, bi_predictions_list)

# BI-MRR 출력
logger.info(f'----------------------------------------------------------------------------')
logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
logger.info('*BI-MRR:{:.4f}'.format(bi_score))
logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
# 검색 한 계슈
#logger.info(f'---------------------------------------------------------------------------')
search_count = 0
for item in bi_ranks:
    if item != 0:
        search_count += 1
    
logger.info('*검색률: {}/{}({:.2f}%)'.format(search_count, len(bi_ranks), (search_count/len(bi_ranks))*100))
logger.info(f'---------------------------------------------------------------------------')

2023-03-27 14:30:08,885 - MRR-sliding - INFO - --------------------------------------------------------------------------
[Kss]: --------------------------------------------------------------------------
2023-03-27 14:30:08,888 - MRR-sliding - INFO - json_file:./data/KorQuAD_v1.0_dev.json
[Kss]: json_file:./data/KorQuAD_v1.0_dev.json
2023-03-27 14:30:08,891 - MRR-sliding - INFO - faiss 인덱싱 방식: max 토큰:40/0(0=코사인 유사도, 1=유클리드 거리)/차원축소:0(128)/ONNX:False
[Kss]: faiss 인덱싱 방식: max 토큰:40/0(0=코사인 유사도, 1=유클리드 거리)/차원축소:0(128)/ONNX:False
2023-03-27 14:30:08,893 - MRR-sliding - INFO - *avg_num:1/search_k:5/query_num:500/out_dimension:0
[Kss]: *avg_num:1/search_k:5/query_num:500/out_dimension:0
2023-03-27 14:30:08,896 - MRR-sliding - INFO - ----------------------------------------------------------------------------
[Kss]: ----------------------------------------------------------------------------
2023-03-27 14:30:08,898 - MRR-sliding - INFO - *BI-ENCODER:bongsoo/klue-sbert-v1
[Kss]: *BI-ENCODER:bo